### Import

In [54]:
import os
import json
import requests
from bs4 import BeautifulSoup

1. Utils

In [55]:
def extract(ancestor, selector, attribute = None, multiple = False):
    if multiple:
        if attribute:
            return [tag[attribute] for tag in ancestor.select(selector)]
        return [tag.get_text() for tag in ancestor.select(selector)] 
    if attribute:
        try:
            return ancestor.select_one(selector)[attribute]
        except TypeError:
            return None
    try:
        return ancestor.select_one(selector).get_text()
    except AttributeError:
        return None

2. Extration of opinions

In [56]:
with open("./cookie.json", "r") as jf:
    headers = json.load(jf)

3. If there are more pages, move to the next page and repert 2 - 6 steps

In [57]:

try:
    next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
except TypeError:
    next_page = None

product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab-reviews_scroll"

all_opinions = []

while next_page:
    response = requests.get(next_page, headers = headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion = {}
            single_opinion["opinion_id"] = opinion["data-entry-id"]
            single_opinion["author"] = extract(opinion, "span.user-post__author-name")
            single_opinion["recommendation"] = extract(opinion,"span.user-post__author-recomendation > em")
            single_opinion["recommendation"] = None
            single_opinion["stars"] = extract(opinion, "span.user-post__score-count")
            single_opinion["content"] = extract(opinion, "div.user-post__text")
            single_opinion["pros"] = extract(opinion,"div.review-feature__item--positive", None, True)
            single_opinion["cons"] = extract(opinion,"div.review-feature__item--negative", None, True)
            single_opinion["vote_yes"] = extract(opinion, "button.vote-yes","data-total-vote")
            single_opinion["vote_no"] = extract(opinion, "button.vote-no","data-total-vote")
            single_opinion["published"] = extract(opinion, "span.user-post__published > time:nth-child(1)","datetime")
            single_opinion["purchased"] = extract(opinion, "span.user-post__published > time:nth-child(2)","datetime")
            all_opinions.append(single_opinion)
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None
    else:
        print("NOT OK")


In [58]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
with open(f"./opinions{product_id}.json","w", encoding = "UTF-8") as jf:
    json.dump(all_opinions, jf, ensure_ascii = False, indent = 4)